![Astrofisica Computacional](../logo.PNG)

---
## Smoothed Particle Hydrodynamics (SPH)


---


### Resumen

Se presenta un modelos simple que ilustra como solucionar las ecuaciones de la hidrodinámica a partir de un modelo de elementos (particulas) suavizado (Smoothed Particle Hydrodynamics SPH) .


---

---

## El Modelo SPH

En el modelo SPH, un fluido es representado por una colección de elementos (partículas) y cada uno de ellos posee ciertas cantidades físicas relevantes como posición, velocidad, masa, densidad, etc.

Como ejemplo particular se considerará un sistema físico descrito por la ecuación de Euler para un fluido ideal sin disipasión,

\begin{equation}
\rho \frac{d\textbf{v}}{dt} = - \boldsymbol{\nabla} P + \textbf{f}
\end{equation}

donde

- $\rho$: densidad
- $\textbf{v}$: velocidad del fluido
- $P$: Presión
- $\textbf{f}$: Fuerzas por unidad de volumen adicionales 

Al considerar un gas ideal, se introduce la ecuación de estado que lo describe $P=P(\rho)$ y al combinarla con la ecuación de energías, se obtiene la ecuación de estado politrópica:

\begin{equation}
P = k\rho^{\gamma} = k\rho^{1+\frac{1}{n}},
\end{equation}

donde $k$ es una constante de proporcionalidad, $\gamma$ es el índice adiabático y $n$ se denomina el índice politrópico.


### El Kernel

Para resolver el sistema, debe evaluarse la aceleración de cada elemento $\frac{d\textbf{v}}{dt}$ y de la ecuación de Euler se observa que, para lograr este objetivo, deben evaluarse algunas derivadas espaciales. En el modelo SPH se considerará la siguiente identidad, valida para cualquier cantidad escalar $\Phi(\textbf{r})$,

\begin{equation}
\Phi(\textbf{r}) = \int \Phi(\textbf{r}') \delta (\textbf{r} - \textbf{r}') d\textbf{r}'
\end{equation}

donde $\delta (\textbf{r})$ es la función Delta de Dirac. Ahora bien, esta relación se modificará introduciendo un *kernel suavizado* $W(\textbf{r};h)$ que reemplazará a la Delta de Dirac. El parámetro $h$ corresponde precidsamente a la escala de longitud de suavizado. El kernel debe ser una función no-negativa, invariante bajo paridad y debe satisfacer las condiciones

\begin{equation}
\int W(\textbf{r};h) d\textbf{r} = 1
\end{equation}

y 

\begin{equation}
\lim_{h\rightarrow 0} W(\textbf{r};h) = \delta(\textbf{r}).
\end{equation}

Dos ejemplos de kernel utilizados en la literatura son:

* **Kernel Gaussiano**

En est caso se tiene la función

\begin{equation}
W(\textbf{r};h) = \left( \frac{1}{h\sqrt{\pi}}\right)^d \exp \left( -\frac{\left| \textbf{r}\right|^2}{h^2}\right)
\end{equation}

donde $d$ es la dimensión del problema. Este kernel es el más paropiado para resolver problemas físicos [[Gingold and Monaghan, 1977]](https://ui.adsabs.harvard.edu/abs/1977MNRAS.181..375G/abstract).

* **Kernel de spline cúbico**

En este caso, la función que actua como kernel es

\begin{equation}
W(\textbf{r};h) = C_d (h) 
\begin{cases}
(2-q)^3 - 4(1-q)^3 & \hspace{1cm} 0\leq q <1 \\
(2-q)^3 & \hspace{1cm} 1\leq q <2 \\
0 & \hspace{1cm}  q\geq 2
\end{cases}
\end{equation}

donde $q = \frac{\left|\textbf{r}\right|}{h}$ y el coeficiente depende de la dimnsionalidad del problema en la forma

\begin{equation}
C_d (h) = 
\begin{cases}
\frac{1}{6h} & \hspace{1cm} d = 1\\
\frac{5}{14\pi h^2} & \hspace{1cm} d = 2\\
\frac{1}{4\pi h^3} & \hspace{1cm} d = 3
\end{cases}
\end{equation}

Este kernel puede disminuir notablemente el tiempo de computo debido a que es una función con soporte compacto (i.e. el conjunto donde no es nula conforma un conjunto cerrado y acotado) y por ello la interacción entre dos particulas mediada por el kernel es simplemente cero si ellas están suficientemente alejadas una de la otra.


---

## Aproximaciones y Derivadas del Kernel Suavizado

La identidad para la función $\Phi(\textbf{r})$ utilizando la Delta de Dirac se aproxima al introducir el kernel suvizado. De esta forma obtenemos la primera aproximación

\begin{equation}
\Phi(\textbf{r}) \approx \Phi^{(1)} (\textbf{r}) = \int \Phi(\textbf{r}´) W(\textbf{r} - \textbf{r}';h) d\textbf{r}'
\end{equation}

Ahora bien, al discretizar el sistema se introduce un siguiente nivel de aproximación, en el cual la integral sobre todo el volumen en el que se encuentra el fluido se reemplaza por una suma de los elementos discretos considerados ($j=1,2,...,N$). Así, se tiene

\begin{equation}
\Phi^{(1)}(\textbf{r}) \approx \Phi^{(2)} (\textbf{r}) = \sum_j \Phi(\textbf{r}_j) W(\textbf{r} - \textbf{r}_j;h) \Delta V_j
\end{equation}

donde $\textbf{r}_j$ representa la posición de la partícula $j$ y $\Delta V_j = \frac{m_j}{\rho_j}$ con $m_j$ la masa del elemento $j$ y $\rho_j$ su densidad.

A partir de estas relaciones se puede construir el gradiente de la función $\Phi(\textbf{r})$ en la forma

\begin{equation}
\boldsymbol{\nabla} \Phi (\textbf{r}) \approx \boldsymbol{\nabla} \Phi^{(2)} (\textbf{r}) = \boldsymbol{\nabla} \sum_j \Phi(\textbf{r}_j) W(\textbf{r} - \textbf{r}_j;h) \Delta V_j =  \sum_j \Phi(\textbf{r}_j) \boldsymbol{\nabla} W(\textbf{r} - \textbf{r}_j;h) \Delta V_j.
\end{equation}

De forma similar, el Laplaciano de $\Phi(\textbf{r})$ será

\begin{equation}
\boldsymbol{\nabla}^2 \Phi (\textbf{r}) \approx \boldsymbol{\nabla}^2 \Phi^{(2)} (\textbf{r}) =  \sum_j \Phi(\textbf{r}_j) \boldsymbol{\nabla}^2 W(\textbf{r} - \textbf{r}_j;h) \Delta V_j
\end{equation}

## Escritura de la Ecuación Diferencial utilizando el Kernel Suavizado

Ya que tenemos las expresiones para las derivadas de una función escalar $\Phi(\textbf{r})$ utilizando el kernel suavizado, podremos escribir la ecuación de Euler en su representación del método SPH. Primero, tenemos la ecuación en la forma

\begin{equation}
\frac{d\textbf{v}}{dt} = - \frac{1}{\rho} \boldsymbol{\nabla} P + \frac{1}{\rho} \textbf{f}.
\end{equation}

El primer término del lado derecho se puede re-escribir como

\begin{equation}
 \frac{1}{\rho} \boldsymbol{\nabla} P = \boldsymbol{\nabla} \left( \frac{P}{\rho}\right) + \frac{P}{\rho^2} \boldsymbol{\nabla} \rho
\end{equation}

Aplicando la representación en términos del kernel suavizado, estos terminos se pueden escribir en la forma

\begin{equation}
  \boldsymbol{\nabla} \left( \frac{P}{\rho}\right) \approx   \sum_j \frac{P_j}{\rho_j} \boldsymbol{\nabla} W(\textbf{r} - \textbf{r}_j;h) \Delta V_j = \sum_j \frac{P_j}{\rho_j} \boldsymbol{\nabla} W(\textbf{r} - \textbf{r}_j;h) \frac{m_j}{\rho_j} = \sum_j m_j\frac{P_j}{\rho_j^2} \boldsymbol{\nabla} W(\textbf{r} - \textbf{r}_j;h) 
\end{equation}

y

\begin{equation}
\frac{P}{\rho^2} \boldsymbol{\nabla} \rho \approx \frac{P}{\rho^2} \sum_j \rho_j \boldsymbol{\nabla} W(\textbf{r} - \textbf{r}_j;h) \Delta V_j = \frac{P}{\rho^2} \sum_j \rho_j \boldsymbol{\nabla} W(\textbf{r} - \textbf{r}_j;h) \frac{m_j}{\rho_j} = \frac{P}{\rho^2} \sum_j m_j \boldsymbol{\nabla} W(\textbf{r} - \textbf{r}_j;h) 
\end{equation}

De esta forma, se tendrá

\begin{equation}
 \frac{1}{\rho} \boldsymbol{\nabla} P = \sum_j m_j\frac{P_j}{\rho_j^2} \boldsymbol{\nabla} W(\textbf{r} - \textbf{r}_j;h) + \frac{P}{\rho^2} \sum_j m_j \boldsymbol{\nabla} W(\textbf{r} - \textbf{r}_j;h) .
\end{equation}

Al reemplazar en la ecuación de Euler se obtiene

\begin{align}
\frac{d\textbf{v}}{dt} = &- \frac{1}{\rho} \boldsymbol{\nabla} P + \frac{1}{\rho} \textbf{f} \\
\frac{d\textbf{v}_i}{dt} = &- \sum_{j\neq i} m_j\frac{P_j}{\rho_j^2} \boldsymbol{\nabla} W(\textbf{r}_i - \textbf{r}_j;h) - \frac{P_i}{\rho_i^2} \sum_{j\neq i} m_j \boldsymbol{\nabla} W(\textbf{r}_i - \textbf{r}_j;h) + \frac{1}{\rho_i} \textbf{f}_i 
\end{align}

donde se ha introducido la condición $j \neq i$ en las sumatorias para evitar la auto-interacción de los elementos. Factorizando, se puede escribir la ecuación fundamental del método SPH,

\begin{align}
\frac{d\textbf{v}_i}{dt} = &- \sum_{j\neq i} m_j \left( \frac{P_j}{\rho_j^2} + \frac{P_i}{\rho_i^2} \right) \boldsymbol{\nabla} W(\textbf{r}_i - \textbf{r}_j;h) + \textbf{a}_i = \textbf{A}_i,
\end{align}

donde la densidad que aparece en estas expresiones puede obtenerse a partir de

\begin{equation}
\rho_i = \sum_j m_j W(\textbf{r}_i - \textbf{r}_j;h).
\end{equation}

y $\textbf{a}_i = \frac{1}{\rho_i} \textbf{f}_i$ representa la aceleración de la partícula $i$ debida a fuerzas externas (gravedad, viscosidad, etc.). En el caso que se modelará, se incluirán dos aceleraciones $\textbf{a}_i = \textbf{g}_i + \textbf{b}_i$. Los términos corresponden a la aceleración gravitacional, $\textbf{g}_i = - (\boldsymbol{\nabla} \phi)_i$ con $\phi$ el potencial gravitacional; y la aceleración debida a una viscosidad modelada en la forma $\textbf{b}_i = -\nu \textbf{v}_i$, con $\nu$ un coeficiente constante.

## Integración Temporal

Para resolver la ecuación de Euler se debe proveer un conjunto de condiciones iniciales, i.e. posiciones y velocidades, $\textbf{r}(0)$ y $\textbf{v}(0)$, para las $N$ partículas. Utilizando esta información, se puede integrar la ecuación diferencial utilizando métodos como Runge-Kutta o Leap Frog.

Usualmente se utiliza el método Leap Frog debido a que es explícito, de segundo orden y simpléctico. Como se presentó en una clase anterior, este algoritmo establece las ecuaciones iterativas para la posición y la velocidad en la forma

\begin{align}
\begin{cases}
\textbf{v}_{n+1/2} &=  \textbf{v}_{n-1/2} +   \textbf{A}_n \Delta t + \mathcal{O}(h^3)\\
\textbf{r}_{n+1} &=  \textbf{r}_n +  \textbf{v}_{n+1/2} \Delta t  + \mathcal{O}(h^3)
\end{cases}
\end{align}

Claramente, este algoritmo no se auto-inicia y por ello se necesita de un calculo previo de la velocidad en (medio)-tiempo anterior al instante inicial. Para este calculo, se utilizara el método de Euler (como es usual).

Una versión modificada del método Leap Frog conocida como la forma "kick-drift-kick" establece un medio avance en la velocidad, luego el avance espcaial y finalmente otro medio avance en la velocidad,

\begin{align}
\begin{cases}
\textbf{v}_{n+1/2} &=  \textbf{v}_{n} +   \textbf{A}_n \frac{\Delta t}{2} \\
\textbf{r}_{n+1} &=  \textbf{r}_n +  \textbf{v}_{n+1/2} \Delta t  \\
\textbf{v}_{n+1} &=  \textbf{v}_{n+1/2} +   \textbf{A}_{n+1} \frac{\Delta t}{2}
\end{cases}
\end{align}


NOTA: en el caso particular que se modelará se puede notar que la aceleración debida a la viscosidad depende de la velocidad, $\textbf{b}_i = -\nu \textbf{v}_i$, lo cual hace que el método no sea completamente explícito. Sin embargo, con la disposición del algorithmo "kick-drift-kick" no hay inconveniente con esta dependencia.

## Longitud Espacial de Suavizado y Tamaño de los Pasos Temporales 


El valor de la escala de suavizado $h$ debe escogerse de tal forma que sea lo suficientemente pequeña para mejorar el tiempo de computo pero lo suficeitnemente grande para que las particulas tengan con quien interactuar. Especialmente en el caso de los kernel con soporte compacto, el valor de $h$ no debe ser demasiado pequeño porque puede tenerse el caso en el que las particulas no interactuen entre si.

De esta forma existen varios criterios para escoger $h$. 
* De acuerdo con [[Gingold and Monaghan, 1977]](https://ui.adsabs.harvard.edu/abs/1977MNRAS.181..375G/abstract) es posible tomar su valor del orden de $\sqrt{\langle \textbf{r}^2\rangle - \langle \textbf{r}\rangle ^2}$, donde $\langle . \rangle$ denota el promedio sobre todas las partículas. 
 
* En [[Monaghan, 1992]](https://ui.adsabs.harvard.edu/abs/1992ARA%26A..30..543M/abstract), se considera un tamaño variable que se determina de tal forma que la bola de radio $h_i$ alrededor de cada particula debe contener $k$-vecinos.


De acuerdo con el análisis de estabilidad realizado por [[Gingold and Monaghan, 1977]](https://ui.adsabs.harvard.edu/abs/1977MNRAS.181..375G/abstract), el tamaño del paso temporal debe satisfacer

\begin{equation}
\Delta t \leq \min \left[ \frac{h}{v_{max}}, \sqrt{\frac{h}{F_{max}}} \right]
\end{equation}

donde $v_{max}$ y $F_{max}$ son la máxima velocidad y la máxima fuerza que siente la partícula en un tiempo dado. Este criteriio puede servir para generar un algoritmo con un paso temporal variable. Sin embargo en el caso particular que se analizará se trabajará con un $\Delta t$ constante y lo suficientemente pequeño para que siempre se cumpla esta condición de estabilidad. Un algoritmo con paso variable utilizando el mismo esquema y aplicado a un sistema mucho mas grande se puede consultar en [[Monaghan, 1992]](https://ui.adsabs.harvard.edu/abs/1992ARA%26A..30..543M/abstract).


---

## Modelo Lineal para la Gravedad

Como un primer modelo (de juguete) se considera la aproximación de una acelaración gravitacional lineal,

\begin{equation}
\textbf{g}_i = - \lambda \textbf{r}_i.
\end{equation}

Este modelo simplificado esta inspirado por el paper [[Monaghan & Price, 2004]](https://academic.oup.com/mnras/article/350/4/1449/986477), en donde se analizan estrellas de juguete 1-dimensionales. Aunque permite ver un comportamiento interesante, el modelo no es completamente realista y luego será modificado para introducir un campo gravitacional mas apropiado.

El valor de la constante $lambda$ se determina al considerar la ecuación de Euler,

\begin{equation}
\frac{d\textbf{v}}{dt} = - \frac{1}{\rho} \boldsymbol{\nabla} P -\nu \textbf{v} - \lambda \textbf{r}
\end{equation}

y considerar el estado de equilibrio que se obtendrá debido a la disipación del termino de viscosidad. En el equilibrio se tendrá $\textbf{v} = 0$ y con ello la ecuación se reduce a

\begin{equation}
0 = - \frac{1}{\rho} \boldsymbol{\nabla} P  - \lambda \textbf{r}.
\end{equation}

Utilizando la ecuación politrópica, se tiene

\begin{align}
\lambda \textbf{r} = &- \frac{1}{\rho} \boldsymbol{\nabla} \left( k \rho ^{1 + \frac{1}{n}}\right) \\
\lambda \textbf{r} = &- \frac{k \left( 1 + \frac{1}{n}\right)}{\rho} \rho^{\frac{1}{n}} \boldsymbol{\nabla} \rho 
\end{align}

Consdierando únicamente dependencia radial, esta ecuación se puede escribir como

\begin{equation}
\lambda r = - k \left( 1 + \frac{1}{n}\right) \rho^{\frac{1}{n}-1} \frac{d\rho}{dr},
\end{equation}

la cual se puede integrar en la forma

\begin{equation}
- k \left( 1 + \frac{1}{n}\right) \int_{\rho (r)} ^{0} \rho^{\frac{1}{n}-1} d\rho = \lambda \int_{r}^{R} r dr
\end{equation}

donde $R$ es el radio de la estrella, en donde $\rho(R) = 0$. De esta forma

\begin{equation}
 k \left( 1 + \frac{1}{n}\right)n \rho (r) ^{\frac{1}{n}}  = \frac{\lambda}{2}\left(R^2 - r^2 \right)
\end{equation}

\begin{equation}
k \left( 1 + n \right) \rho (r) ^{\frac{1}{n}}  = \frac{\lambda}{2}\left(R^2 - r^2 \right)
\end{equation}

de donde se obtiene el perfil de densidad

\begin{equation}
 \rho (r) = \left[ \frac{\lambda}{2k \left( 1 + n \right)}\left(R^2 - r^2 \right) \right]^n .
\end{equation}

Suponiendo que la estrella tiene un radio $R$ y una masa total $M$, se tiene la relación

\begin{equation}
M = \int \rho (r) d^d \textbf{r}
\end{equation}

donde $d$ es la dimensión del espacio. De esta relación se obtiene el valor de $\lambda$ como

\begin{equation}
\lambda = 
\begin{cases}
\frac{2k\pi^{-\frac{1}{n}}}{M} \left(\frac{M(1+n)}{R^2} \right)^{1+\frac{1}{n}} \hspace{1cm} d=2 \\
\frac{2k(1+n)\pi^{-\frac{3}{2n}}}{R^2} \left(\frac{M\Gamma (\frac{5}{2}+n)}{R^3 \Gamma (1+n)} \right)^{\frac{1}{n}} \hspace{1cm} d=3
\end{cases}
\end{equation}